In [1]:
# !pip install geopandas
import geopandas as gpd
import pandas as pd


In [2]:
# save api responses as variables
with open("api_responses_1hour.txt") as f:
    exec(f.read())

In [3]:
# function to add minute and label properties to api data for later visualizations
def add_travel_time_properties(name, minutes, label):
    isochrone = eval(name)
    for feature in isochrone["features"]:
        feature["properties"]["travel_time_min"] = minutes
        feature["properties"]["travel_time_label"] = label
        
        feature["properties"].pop("search_id", None) # remove 'search_id' since they're all "0" from API

    return {"name": name, "travel_time_min": minutes, "travel_time_label": label, "data": isochrone}

# save list of isocrhone variable names
variable_names = [f"isochrone{i:03d}" for i in range(1, 61)]
# save list of travel times as integers
travel_times = list(range(1, 61))
# save list of travel time labels
travel_labels = [f"{i:03d}_min" for i in travel_times]

# save all data into dictionary
isochrone_dict = {name: add_travel_time_properties(name, minutes, label) for name, minutes, label in zip(variable_names, travel_times, travel_labels)}

In [4]:
# set path to your output GeoPackage
#gpkg_file_name = "Paris_3hours_1min-isochrones.gpkg"
gpkg_file_name = "Paris_1hr_1min-isochrones.gpkg"

# loop through dictionary and save each isochrone as a layer
for name, entry in isochrone_dict.items():
    # convert the GeoJSON dict to a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(entry["data"]["features"], crs="EPSG:4326")

    # optional: add global metadata columns for convenience
    gdf["travel_time_min"] = entry["travel_time_min"]
    gdf["travel_time_label"] = entry["travel_time_label"]

    # save to GPKG (each layer named after the isochrone)
    gdf.to_file(gpkg_file_name, layer=name, driver="GPKG")

In [5]:
# save all into one layer

# Collect all features into one GeoDataFrame
all_gdfs = []

for name, entry in isochrone_dict.items():
    gdf = gpd.GeoDataFrame.from_features(entry["data"]["features"], crs="EPSG:4326")
    gdf["travel_time_min"] = entry["travel_time_min"]
    gdf["travel_time_label"] = entry["travel_time_label"]
    gdf["layer_name"] = name  # optional: keep source layer reference
    all_gdfs.append(gdf)

# Combine into one GeoDataFrame
gdf_all = pd.concat(all_gdfs, ignore_index=True)

# order by travel time
gdf_all = gdf_all.sort_values("travel_time_min").reset_index(drop=True)
gdf_all.set_crs("EPSG:4326", inplace=True)


# save as one layer
output_path = "Paris_1hr_isochrones_combined.gpkg"
gdf_all.to_file(output_path, layer="all_isochrones", driver="GPKG")
